## Part 0. Package setting
The way to do this is to manually check how to install the packages in your environment. 
this worked on collab but I installed the packages manually in my machine, the porcedure to get them will likely ghe different depending on your environment

In [ ]:
#  Install necessary libraries
!pip install transformers torch torchaudio pyaudio pydub huggingface_hub librosa soundfile

In [ ]:
# I seems that some libraries are needed in order to install pyaudio.
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio pydub

In [38]:
# Import libraries
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor, WhisperTokenizer
import pyaudio
import wave
import librosa
import soundfile as sf
from pydub import AudioSegment
from IPython.display import Audio
import os
from scipy.signal import resample

In [36]:
# This part seems necessary for later

import evaluate
from jiwer import cer

def down_sample_audio(audio_original, original_sample_rate):
    target_sample_rate = 16000

    # Calculate the number of samples for the target sample rate
    num_samples = int(len(audio_original) * target_sample_rate / original_sample_rate)

    # Resample the audio array to the target sample rate
    downsampled_audio = resample(audio_original, num_samples)

    return downsampled_audio

## Part 1: Loading model and evaluating

In [49]:
# Load every part of the small model separately

MODEL_REPO = "SchindleriaPraematurus/whisper-tiny-gn-finetuned" # Your repository
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


SMALL_TOKENIZER_COMMIT_HASH = "6cd67df1b6923465434cee5bb80aadfe7a8d74ab"  # Commit for "Upload tokenizer"
SMALL_FEATURE_EXTRACTOR_COMMIT_HASH = "a7f430d0d46753348597fb241734d341c48fc69b" # Commit for "Upload feature_extractor"
SMALL_MODEL_COMMIT_HASH = "06294d5a001f1f32290e98cf9a4a6006f1fc394a" # Commit for "Upload fine-tuned Whisper-small"


In [52]:
# Movin to CPU because I ran our of ram LOL (Change this)
DEVICE='cpu' 

In [53]:


print(f"Attempting to load 'small' version components from specific commits:")
print(f" - Tokenizer commit: {SMALL_TOKENIZER_COMMIT_HASH}")
print(f" - Feature Extractor commit: {SMALL_FEATURE_EXTRACTOR_COMMIT_HASH}")
print(f" - Model commit: {SMALL_MODEL_COMMIT_HASH}")

tokenizer_small = None
feature_extractor_small = None
model_small = None
processor_small = None

try:
    # Load Tokenizer
    # These settings are usually saved in tokenizer_config.json.
    print(f"\nLoading small tokenizer from revision: {SMALL_TOKENIZER_COMMIT_HASH}...")
    tokenizer_small = WhisperTokenizer.from_pretrained(
        MODEL_REPO,
        revision=SMALL_TOKENIZER_COMMIT_HASH
    )
    print("Successfully loaded 'small' tokenizer.")

    # Load Feature Extractor
    print(f"\nLoading small feature extractor from revision: {SMALL_FEATURE_EXTRACTOR_COMMIT_HASH}...")
    feature_extractor_small = WhisperFeatureExtractor.from_pretrained(
        MODEL_REPO,
        revision=SMALL_FEATURE_EXTRACTOR_COMMIT_HASH
    )
    print("Successfully loaded 'small' feature extractor.")

    # Load Model
    print(f"\nLoading small model from revision: {SMALL_MODEL_COMMIT_HASH}...")
    model_small = WhisperForConditionalGeneration.from_pretrained(
        MODEL_REPO,
        revision=SMALL_MODEL_COMMIT_HASH
    ).to(DEVICE)
    model_small.eval()
    print("Successfully loaded 'small' model.")

    # Manually create the WhisperProcessor instance
    if tokenizer_small and feature_extractor_small:
        processor_small = WhisperProcessor(feature_extractor=feature_extractor_small, tokenizer=tokenizer_small)
        print("\nSuccessfully created 'small' WhisperProcessor manually.")
    else:
        print("\nCould not create 'small' processor because tokenizer or feature extractor failed to load.")

except Exception as e:
    print(f"\nAn error occurred during loading of 'small' components: {e}")
    print("Please double-check the following:")
    print("1. All three commit hashes are correct and point to the respective uploads for the 'small' version.")
    print(f"2. The tokenizer files (vocab.json, tokenizer_config.json, etc.) are indeed present at commit {SMALL_TOKENIZER_COMMIT_HASH}.")
    print(f"3. The feature extractor's preprocessor_config.json is present at commit {SMALL_FEATURE_EXTRACTOR_COMMIT_HASH}.")
    print(f"4. The model files (pytorch_model.bin, config.json) are present at commit {SMALL_MODEL_COMMIT_HASH}.")

if model_small and processor_small:
    print(f"\nSuccessfully prepared 'small' model (from multiple commits) for comparison.")
    # You can now use model_small and processor_small for predictions
else:
    print(f"\nFailed to fully load the 'small' model and processor. Check error messages above.")

Attempting to load 'small' version components from specific commits:
 - Tokenizer commit: 6cd67df1b6923465434cee5bb80aadfe7a8d74ab
 - Feature Extractor commit: a7f430d0d46753348597fb241734d341c48fc69b
 - Model commit: 06294d5a001f1f32290e98cf9a4a6006f1fc394a

Loading small tokenizer from revision: 6cd67df1b6923465434cee5bb80aadfe7a8d74ab...
Successfully loaded 'small' tokenizer.

Loading small feature extractor from revision: a7f430d0d46753348597fb241734d341c48fc69b...
Successfully loaded 'small' feature extractor.

Loading small model from revision: 06294d5a001f1f32290e98cf9a4a6006f1fc394a...
Successfully loaded 'small' model.

Successfully created 'small' WhisperProcessor manually.

Successfully prepared 'small' model (from multiple commits) for comparison.


## Part 2. Getting audio as input

In [12]:
# Cell 4: Record audio for a fixed duration
FORMAT = pyaudio.paInt16  # Audio format (16-bit PCM)
CHANNELS = 1              # Number of audio channels (1 for mono, 2 for stereo)
RATE = 16000              # Sample rate (Whisper expects 16kHz)
CHUNK = 1024              # Number of frames per buffer
RECORD_SECONDS = 5        # Duration of recording in seconds
WAV_FILENAME = "recorded_audio.wav"
MP3_FILENAME = "recorded_audio.mp3"

audio = pyaudio.PyAudio()

In [17]:

# Start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

print("Recording...")
frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Finished recording.")

# Stop Recording
stream.stop_stream()
stream.close()
audio.terminate()

# Save the recorded data as a WAV file
wf = wave.open(WAV_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

print(f"Audio saved as {WAV_FILENAME}")

# Convert WAV to MP3 using pydub
try:
    sound = AudioSegment.from_wav(WAV_FILENAME)
    sound.export(MP3_FILENAME, format="mp3")
    print(f"Converted to {MP3_FILENAME}")
    os.remove(WAV_FILENAME) # Optional: remove the intermediate WAV file
    # Display the audio player in the notebook
    display(Audio(MP3_FILENAME))
except Exception as e:
    print(f"Could not convert to MP3. Make sure ffmpeg is installed and in your PATH. Error: {e}")
    print("You can still use the WAV file for prediction if MP3 conversion fails, but ensure your prediction block loads the correct file type.")

Recording...
Finished recording.
Audio saved as recorded_audio.wav
Converted to recorded_audio.mp3


In [14]:
# Cell 5: Record audio (stop with Enter key in console - more for script, but can work)
# This method will record until you press Enter in the console where Jupyter is running.
# Note: In some environments (like Colab without specific widgets), direct key press detection is hard.
# A simpler alternative is just a longer fixed duration.

FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000 # Whisper expects 16kHz
CHUNK = 1024
WAV_FILENAME_MANUAL = "recorded_audio_manual.wav"
MP3_FILENAME_MANUAL = "recorded_audio_manual.mp3"

audio = pyaudio.PyAudio()


In [ ]:
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

print("Recording... Press Enter in the console running Jupyter to stop.")
frames = []
# This input() will block, so recording happens while it waits.
# This isn't a perfect "stop recording on keypress" but a simple way to gate it.
# For true keypress stop, you'd need a more complex GUI or threading.
try:
    while True: # A loop to simulate continuous recording until input
        data = stream.read(CHUNK)
        frames.append(data)
        # This is a crude way to check for input without blocking read too much
        # It's not ideal. A better way is to record fixed long duration or use threads.
        # For this example, let's make it simpler: Record for a max duration or until input.
        # The input() below is the primary stop.
except KeyboardInterrupt: # If you press Ctrl+C in console
    print("Recording stopped by user.")
except Exception as e:
    pass # Handle other potential stream errors

# The following is a simpler approach for notebooks:
# Ask user to press enter, then record for a few seconds.
# Or, just use a fixed duration like Method 1.

# Let's refine Method 2 for a notebook: record after user hits enter.
print("Prepare to record. Press Enter to start recording for 10 seconds.")
input() # Wait for user to press Enter

print("Recording for 10 seconds...")
frames = []
for i in range(0, int(RATE / CHUNK * 10)): # Record for 10 seconds
    data = stream.read(CHUNK)
    frames.append(data)
print("Finished recording.")

stream.stop_stream()
stream.close()
audio.terminate()

wf = wave.open(WAV_FILENAME_MANUAL, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()
print(f"Audio saved as {WAV_FILENAME_MANUAL}")

try:
    sound = AudioSegment.from_wav(WAV_FILENAME_MANUAL)
    sound.export(MP3_FILENAME_MANUAL, format="mp3")
    print(f"Converted to {MP3_FILENAME_MANUAL}")
    os.remove(WAV_FILENAME_MANUAL)
    display(Audio(MP3_FILENAME_MANUAL))
except Exception as e:
    print(f"Could not convert to MP3: {e}. Using WAV.")
    MP3_FILENAME_MANUAL = WAV_FILENAME_MANUAL # Fallback to WAV if MP3 fails

## Part 3. Geting predictions with new model

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=tokenizer,
    feature_extractor=feature_extractor,
    chunk_length_s=30,     # 30s chunks
    stride_length_s=10,    # 10s overlap
    device='cuda',
)


In [ ]:
file_path = "recorded_audio.mp3"
result = pipe(file_path)
result

## Part 4. Comparing model sizes

In [30]:
# Cell 7: Load the 'tiny' version components using their specific commit hashes

MODEL_REPO = "SchindleriaPraematurus/whisper-tiny-gn-finetuned" # Your repository
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


TINY_TOKENIZER_COMMIT_HASH = "5d8e8d409cb195deb958c6733939dae432db2a52"  # Commit for "Upload tokenizer"
TINY_FEATURE_EXTRACTOR_COMMIT_HASH = "612f9f357ea4677f69fcfae2ffaa4c55534a3ac2" # Commit for "Upload feature_extractor"
TINY_MODEL_COMMIT_HASH = "712fa109774ac1a772be3c29b718785afa865a9e" # Commit for "Upload fine-tuned Whisper-tiny"


In [31]:


print(f"Attempting to load 'tiny' version components from specific commits:")
print(f" - Tokenizer commit: {TINY_TOKENIZER_COMMIT_HASH}")
print(f" - Feature Extractor commit: {TINY_FEATURE_EXTRACTOR_COMMIT_HASH}")
print(f" - Model commit: {TINY_MODEL_COMMIT_HASH}")

tokenizer_tiny = None
feature_extractor_tiny = None
model_tiny = None
processor_tiny = None

try:
    # Load Tokenizer
    # Your training script loaded the initial tokenizer with language="spanish", task="transcribe".
    # These settings are usually saved in tokenizer_config.json.
    # Try loading without explicit language/task first, as it should use the saved config.
    print(f"\nLoading tiny tokenizer from revision: {TINY_TOKENIZER_COMMIT_HASH}...")
    tokenizer_tiny = WhisperTokenizer.from_pretrained(
        MODEL_REPO,
        revision=TINY_TOKENIZER_COMMIT_HASH
    )
    print("Successfully loaded 'tiny' tokenizer.")

    # Load Feature Extractor
    print(f"\nLoading tiny feature extractor from revision: {TINY_FEATURE_EXTRACTOR_COMMIT_HASH}...")
    feature_extractor_tiny = WhisperFeatureExtractor.from_pretrained(
        MODEL_REPO,
        revision=TINY_FEATURE_EXTRACTOR_COMMIT_HASH
    )
    print("Successfully loaded 'tiny' feature extractor.")

    # Load Model
    print(f"\nLoading tiny model from revision: {TINY_MODEL_COMMIT_HASH}...")
    model_tiny = WhisperForConditionalGeneration.from_pretrained(
        MODEL_REPO,
        revision=TINY_MODEL_COMMIT_HASH
    ).to(DEVICE)
    model_tiny.eval()
    print("Successfully loaded 'tiny' model.")

    # Manually create the WhisperProcessor instance
    if tokenizer_tiny and feature_extractor_tiny:
        processor_tiny = WhisperProcessor(feature_extractor=feature_extractor_tiny, tokenizer=tokenizer_tiny)
        print("\nSuccessfully created 'tiny' WhisperProcessor manually.")
    else:
        print("\nCould not create 'tiny' processor because tokenizer or feature extractor failed to load.")

except Exception as e:
    print(f"\nAn error occurred during loading of 'tiny' components: {e}")
    print("Please double-check the following:")
    print("1. All three commit hashes are correct and point to the respective uploads for the 'tiny' version.")
    print(f"2. The tokenizer files (vocab.json, tokenizer_config.json, etc.) are indeed present at commit {TINY_TOKENIZER_COMMIT_HASH}.")
    print(f"3. The feature extractor's preprocessor_config.json is present at commit {TINY_FEATURE_EXTRACTOR_COMMIT_HASH}.")
    print(f"4. The model files (pytorch_model.bin, config.json) are present at commit {TINY_MODEL_COMMIT_HASH}.")

if model_tiny and processor_tiny:
    print(f"\nSuccessfully prepared 'tiny' model (from multiple commits) for comparison.")
    # You can now use model_tiny and processor_tiny for predictions
else:
    print(f"\nFailed to fully load the 'tiny' model and processor. Check error messages above.")

Attempting to load 'tiny' version components from specific commits:
 - Tokenizer commit: 5d8e8d409cb195deb958c6733939dae432db2a52
 - Feature Extractor commit: 612f9f357ea4677f69fcfae2ffaa4c55534a3ac2
 - Model commit: 712fa109774ac1a772be3c29b718785afa865a9e

Loading tiny tokenizer from revision: 5d8e8d409cb195deb958c6733939dae432db2a52...


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

...

Successfully loaded 'tiny' feature extractor.

Loading tiny model from revision: 712fa109774ac1a772be3c29b718785afa865a9e...


config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Successfully loaded 'tiny' model.

Successfully created 'tiny' WhisperProcessor manually.

Successfully prepared 'tiny' model (from multiple commits) for comparison.


In [41]:
model_tiny.generation_config.forced_decoder_ids = None

model_tiny.generation_config.language = "spanish"
model_tiny.generation_config.task = "transcribe"


gen_config_for_tiny = model_tiny.generation_config

In [44]:
from datasets import load_dataset
model_tiny.eval()


test_data = load_dataset("mozilla-foundation/common_voice_17_0", "gn", split="test")

for idx in range(10):

    target = test_data[idx]['sentence']
    audio_original = test_data[idx]['audio']['array']
    original_sample_rate = test_data[idx]['audio']['sampling_rate']

    audio_16000 = down_sample_audio(audio_original, original_sample_rate)

    input_feature = feature_extractor_tiny(raw_speech=audio_16000,
                                    sampling_rate=16000,
                                    return_tensors='pt').input_features

    with torch.no_grad():
        op = model_tiny.generate(input_feature.to('cuda'), generation_config=gen_config_for_tiny)


    text_pred =  tokenizer_tiny.batch_decode(op,skip_special_tokens=True )[0]

    print(f'-------{idx}------')
    print(f'true : {target} \npred : {text_pred}')
    print('\n ')

-------0------
true : omba'apóva ñanduti iporãve hag̃ua 
pred : Omba'apóva ñanduti iporãve hag̃ua.

 
-------1------
true : Ñúme katu ynambu oĩva'ekue. 
pred : Ñome katu ymamboy'íva'ekue.

 
-------2------
true : Heta ára ndaje ohasa asy. 
pred : Heta ára ndaje ohasa'asy.

 
-------3------
true : ¡Pepracticavéke! 
pred : Peperatĩ kavéke.

 
-------4------
true : sa'íma oĩ ka'aguy 
pred : sa'íma oĩ ka'aguý

 
-------5------
true : ha ijykére mitãrusu roky 
pred : ha ijykére mitãrusu roky

 
-------6------
true : iporã umíva ojapokuaa 
pred : Iporã umíva ojapokuaa.

 
-------7------
true : Mitã ojapo'ỹva isy ha itúva ojeruréva chupekuéra. 
pred : Mitã ojapo'ỹva isy ha itúva ojerureva chupekuéra.

 
-------8------
true : Oiménepiko ojeráma 
pred : Oiméne piko ojeráma.

 
-------9------
true : Hetaitereíko ihénte la abuela 
pred : Hetaiterei ko'i hénte la abuela.

 


In [70]:
model_small.generation_config.forced_decoder_ids = None

model_small.generation_config.language = "spanish"
model_small.generation_config.task = "transcribe"


gen_config_for_small = model_small.generation_config


In [73]:
model_small.eval()

for idx in range(10):

    target = test_data[idx]['sentence']
    audio_original = test_data[idx]['audio']['array']
    original_sample_rate = test_data[idx]['audio']['sampling_rate']

    audio_16000 = down_sample_audio(audio_original, original_sample_rate)

    input_feature = feature_extractor_small(raw_speech=audio_16000,
                                    sampling_rate=16000,
                                    return_tensors='pt').input_features

    with torch.no_grad():
        op = model_small.generate(input_feature.to('cpu'), generation_config=gen_config_for_small)


    text_pred =  tokenizer_small.batch_decode(op,skip_special_tokens=True )[0]

    print(f'-------{idx}------')
    print(f'true : {target} \npred : {text_pred}')
    print('\n ')

-------0------
true : omba'apóva ñanduti iporãve hag̃ua 
pred : omba'apóva ñanduti iporãve hag̃ua

 
-------1------
true : Ñúme katu ynambu oĩva'ekue. 
pred : Ñúme katu ynambu oĩva'ekue.

 
-------2------
true : Heta ára ndaje ohasa asy. 
pred : Heta ára ndaje ohasa asy.

 
-------3------
true : ¡Pepracticavéke! 
pred : ¡Pepracticavéke!

 
-------4------
true : sa'íma oĩ ka'aguy 
pred : sa'íma oĩ ka'aguy

 
-------5------
true : ha ijykére mitãrusu roky 
pred : ha ijykére mitãrusu roky

 
-------6------
true : iporã umíva ojapokuaa 
pred : iporã umíva ojapokuaa

 
-------7------
true : Mitã ojapo'ỹva isy ha itúva ojeruréva chupekuéra. 
pred : Mitã ojapo'ỹva isy ha itúva ojeruréva chupekuéra.

 
-------8------
true : Oiménepiko ojeráma 
pred : Oiménepiko ojeráma

 
-------9------
true : Hetaitereíko ihénte la abuela 
pred : Hetaitereíko ihénte la abuela

 
